In [1]:
%load_ext lab_black
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

1. To test whether the number of police officers certified per month (from 6-2015 to 5-2021, accumulative) is negatively correlated with cases with use of force per month.
2. To test whether the number of police officers certified per month (from 6-2015 to 5-2021, accumulative) is negatively correlated with cases with use of force per month on crisis situations only.
3. Test 1 and 2 per sector/precinct/beat (simpson's paradox)

In [44]:
df_training = pd.read_csv("../data/processed/MVAtrain_cleaned.csv")

In [45]:
df_training.head(10)

,File Num,GO Num,CIT Certified Ind,Incident Type,Latitude,Longitude,Officer Gender,Officer Race,Officer Serial Num,Officer Title (as of Incident),Precinct,Sector,Subject Gender,Subject Race,Occurred Date
0,2015UOF-0336,20130000117995,N,Level 1 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,F,Asian,2015-03-03
1,2015UOF-0540,20130000117995,N,Level 1 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,F,Not Specified,2015-04-21
2,2015UOF-0335,20130000117995,N,Level 1 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,M,Asian,2015-03-03
3,2015UOF-0334,20130000117995,N,Level 2 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,M,Asian,2015-03-03
4,2014UOF-1166,20130000458895,N,Level 1 - Use of Force,NaN,NaN,M,Hispanic or Latino,NaN,NaN,X,NaN,M,Black or African American,2014-10-15
5,2014UOF-1165,20130000458895,N,Level 1 - Use of Force,NaN,NaN,M,Not Specified,NaN,NaN,X,NaN,M,Black or African American,2014-10-15
6,2014UOF-0365,20140000129621,N,Level 2 - Use of Force,NaN,NaN,M,Hispanic or Latino,NaN,NaN,X,NaN,M,White,2014-04-28
7,2014UOF-0367,20140000129621,N,Level 2 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,M,White,2014-04-28
8,2014UOF-0366,20140000129621,N,Level 2 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,M,White,2014-04-28
9,2014UOF-0694,20140000137215,N,Level 1 - Use of Force,NaN,NaN,M,White,NaN,NaN,X,NaN,N,Not Specified,2014-05-04


In [2]:
df = pd.read_csv("../data/processed/MVAcrisis_cleaned.csv")

In [3]:
df.head(10)

,Behavioral Crisis Num,CAD Event ID,Disposition,Behavior,OffenseIncident,Age,Gender,Race,Techniques,UoF,Weapons,Date
0,43653,2015000159907,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-15
1,43719,2015000160408,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-16
2,43845,2015000160465,RESOURCES OFFERED/DECLINED,-,-,-1,-,-,-,NO,-,2015-05-16
3,43992,2015000159631,GERIATRIC REGIONAL ASSESSMENT TEAM,-,-,-1,-,-,-,NO,-,2015-05-15
4,44102,-1,VOLUNTARY COMMITTAL,-,-,-1,-,-,-,NO,-,2015-05-16
5,44320,2015000162604,VOLUNTARY COMMITTAL,-,-,-1,-,-,-,NO,-,2015-05-18
6,44403,2015000161923,MOBILE CRISIS TEAM,-,-,-1,-,-,-,NO,-,2015-05-17
7,44431,2015000161916,CHRONIC COMPLAINT,-,-,-1,-,-,-,NO,-,2015-05-17
8,44432,2015000161916,-,-,-,-1,-,-,-,NO,-,2015-05-17
9,44440,-1,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-17


In [17]:
df["Date"] = pd.to_datetime(df["Date"])
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year

In [7]:
df.head(10)

,Behavioral Crisis Num,CAD Event ID,Disposition,Behavior,OffenseIncident,Age,Gender,Race,Techniques,UoF,Weapons,Date,Month
0,43653,2015000159907,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-15,5
1,43719,2015000160408,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-16,5
2,43845,2015000160465,RESOURCES OFFERED/DECLINED,-,-,-1,-,-,-,NO,-,2015-05-16,5
3,43992,2015000159631,GERIATRIC REGIONAL ASSESSMENT TEAM,-,-,-1,-,-,-,NO,-,2015-05-15,5
4,44102,-1,VOLUNTARY COMMITTAL,-,-,-1,-,-,-,NO,-,2015-05-16,5
5,44320,2015000162604,VOLUNTARY COMMITTAL,-,-,-1,-,-,-,NO,-,2015-05-18,5
6,44403,2015000161923,MOBILE CRISIS TEAM,-,-,-1,-,-,-,NO,-,2015-05-17,5
7,44431,2015000161916,CHRONIC COMPLAINT,-,-,-1,-,-,-,NO,-,2015-05-17,5
8,44432,2015000161916,-,-,-,-1,-,-,-,NO,-,2015-05-17,5
9,44440,-1,NO ACTION POSSIBLE/NECESSARY/UNABLE TO CONTACT,-,-,-1,-,-,-,NO,-,2015-05-17,5


In [10]:
df.drop(df[df["CAD Event ID"] == -1].index, inplace=True)

In [11]:
df.shape

(93912, 13)

In [12]:
len(df["CAD Event ID"].unique())

55651

In [37]:
df.loc[df["CAD Event ID"] == 2015000161916].sort_values(
    by=["CAD Event ID", "Behavioral Crisis Num"], ascending=[True, False]
).drop_duplicates(subset="CAD Event ID", keep="first")

,Behavioral Crisis Num,CAD Event ID,Disposition,Behavior,OffenseIncident,Age,Gender,Race,Techniques,UoF,Weapons,Date,Month,Year
8,44432,2015000161916,-,-,-,-1,-,-,-,NO,-,2015-05-17,5,2015


In [42]:
var_df = (
    df[["CAD Event ID", "UoF", "Month", "Year"]]
    .drop_duplicates(subset="CAD Event ID")
    .reset_index(drop=True)
)

In [43]:
var_df.groupby(["Year", "Month", "UoF"]).size().to_frame()

0
Year Month UoF     
2015 5     NO   305
           YES    9
     6     NO   664
           YES   16
     7     NO   666
...             ...
2021 4     YES   11
     5     NO   906
           YES    7
     6     NO   214
           YES    1

[148 rows x 1 columns]

In [16]:
df.columns

Index(['Behavioral Crisis Num', 'CAD Event ID', 'Disposition', 'Behavior',
       'OffenseIncident', 'Age', 'Gender', 'Race', 'Techniques', 'UoF',
       'Weapons', 'Date', 'Month'],
      dtype='object')

In [15]:
df[["CAD Event ID", "UoF", "Month","Year"]]

KeyError: "['Year'] not in index"